## Segmenting and Clustering Neighborhoods in Toronto

Importing all libraris I need in this assigment:

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: d

In [2]:
!pip install lxml

     |████████████████████████████████| 5.5MB 5.9MB/s eta 0:00:01


In [3]:
from lxml import html

#### Question 1:

the following part is given for readiing the table from Wikipedia

In [16]:
df=pd.read_html(r"https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [23]:
df_toronto=df[0]

In [24]:
df_toronto.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


for removing rows with Not assigned on the "Borough" column: 

In [27]:
df_toronto=df_toronto[df_toronto.Borough!='Not assigned']
df_toronto = df_toronto.reset_index(drop=True)

In [28]:
df_toronto.head(5)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


for replacing data 'Not assigned' in the "Neighborhood" column with data from the "Borough" column: 

In [29]:
df_toronto.Neighbourhood = np.where(df_toronto.Neighbourhood != 'Not assigned',df_toronto.Neighbourhood,df_toronto.Borough)
df_toronto.head(12)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


print the number of rows of your dataframe:

In [45]:
df.shape

(103, 3)

In [5]:
!pip install geocoder


     |████████████████████████████████| 102kB 7.4MB/s ta 0:00:011
     |████████████████████████████████| 829kB 7.6MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


In [6]:
>>> import geocoder


In [19]:
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(df[['Postal Code']]))
  lat_lng_coords = g.latlng

KeyboardInterrupt: 

In [117]:
 df_coor=pd.read_csv(r'http://cocl.us/Geospatial_data')

In [128]:
df_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [191]:
f2 = {'Longitude': range(df.shape[0]+1), 'Latitude': range(df.shape[0]+1)}
df2 = pd.DataFrame(data=f2)
for i in range(df.shape[0]+1):
    for j in range(df_coor.shape[0]+1):
        if df.iloc[i]['Postal Code']== df_coor.iloc[j]['Postal Code']:
            #df.iloc[i]['Longitude']= df_coor.iloc[j]['Longitude']
        #    print(df_coor.iloc[j]['Longitude'])
            df2.iloc[i][['Longitude','Latitude']]=df_coor.iloc[j][['Longitude','Latitude']].to_list()
         #   df.insert(i, 'Longitude', df_coor.iloc[j]['Longitude'])
     #       df.iloc[i]['Latitude']= df_coor.iloc[j]['Latitude']

In [212]:
R=df_coor.iloc[0][['Longitude','Latitude']].to_list()
R

[-79.19435340000001, 43.806686299999996]

In [221]:
type(R)

list

In [226]:
 type(df2.iloc[i][1])

numpy.int64

In [143]:
df.append({'Longitude': df_coor.iloc[j]['Longitude']},ignore_index=True)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,NaN
1,M4A,North York,Victoria Village,43.753259,NaN
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.753259,NaN
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.753259,NaN
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.753259,NaN
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.753259,NaN
6,M1B,Scarborough,"Malvern, Rouge",43.753259,NaN
7,M3B,North York,Don Mills,43.753259,NaN
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.753259,NaN
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.753259,NaN
